In [4]:
import pandas as pd
import json

C:\Users\Will\anaconda3\envs\deep_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Will\anaconda3\envs\deep_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Will\anaconda3\envs\deep_learning\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [5]:
HAP_df = pd.read_parquet('data/pricing_data/avgHighPrice.parquet.gzip')
LAP_df = pd.read_parquet('data/pricing_data/avgLowPrice.parquet.gzip')
HAV_df = pd.read_parquet('data/pricing_data/highPriceVolume.parquet.gzip')
LAV_df = pd.read_parquet('data/pricing_data/lowPriceVolume.parquet.gzip')

In [6]:
with open("data/name_dicts/ID_to_name.json","r") as f:
    id2name=json.load(f)
with open("data/name_dicts/name_to_ID.json","r") as f:
    name2id=json.load(f)

In [7]:
weighted_avg = ((HAP_df.values*HAV_df.values)+(LAP_df.values*LAV_df.values))/(LAV_df.values+HAV_df.values)
df = pd.DataFrame(columns = HAP_df.columns, data = weighted_avg)
totv = pd.DataFrame(columns = HAP_df.columns, data = (LAV_df.values+HAV_df.values)) #a dataframe of volume traded that day
df = df.interpolate(method = "linear")

In [8]:
prices = df[str(name2id["Abyssal whip"])].to_list() #lists of time and value
times = df["timestamps"].to_list()